# Coding Attention Mechanism

# 3.3 Attending to different parts of the input with self_attention

# 3.3.1 A simple self_attention mechanism without trainable weights

In [1]:
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.10], # 0ne
        [0.05, 0.80, 0.55]  # step
    ]
)

In [2]:
input_query = inputs[1]
input_query


tensor([0.5500, 0.8700, 0.6600])

In [3]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [4]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [69]:
res = 0.
i = 0
for idx, ele  in enumerate(inputs[i]):
  res+= inputs[i][idx] * input_query[idx]



res



tensor(0.9544)

In [6]:
i = 3
res = torch.dot(inputs[i], input_query)
res

tensor(0.8434)

In [7]:
inputs.shape[0]

6

In [62]:
torch.zeros(inputs.shape[0])

tensor([0., 0., 0., 0., 0., 0.])

In [9]:
query = inputs[2]
attn_scores_2 = torch.zeros(inputs.shape[0])
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i,input_query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [10]:
attn_weights_2 = attn_scores_2 / attn_scores_2.sum()
print(attn_weights_2)
attn_weights_2.sum()

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


tensor(1.0000)

In [72]:
def softmax_naive(x):
  return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_naive(attn_scores_2)

tensor([0.1401, 0.2507, 0.2406, 0.1157, 0.0687, 0.1842],
       grad_fn=<DivBackward0>)

In [12]:
attn_weights_2= torch.softmax(attn_scores_2, dim=0)

attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [64]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i, X_i in enumerate(inputs):
  print(f"{attn_weights_2[i]} ---> {inputs[i]}")
  context_vec_2 += attn_weights_2[i] * X_i

context_vec_2



0.15001951158046722 ---> tensor([0.4300, 0.1500, 0.8900])
0.22638379037380219 ---> tensor([0.5500, 0.8700, 0.6600])
0.2198716253042221 ---> tensor([0.5700, 0.8500, 0.6400])
0.131070077419281 ---> tensor([0.2200, 0.5800, 0.3300])
0.09062890708446503 ---> tensor([0.7700, 0.2500, 0.1000])
0.18202611804008484 ---> tensor([0.0500, 0.8000, 0.5500])


tensor([0.4221, 0.6506, 0.5761], grad_fn=<AddBackward0>)

In [73]:
inputs.shape

torch.Size([6, 3])

In [74]:
context_vec_2.shape

torch.Size([3])

In [14]:
for i, x_i in enumerate(inputs):
  print(i,inputs[i])

0 tensor([0.4300, 0.1500, 0.8900])
1 tensor([0.5500, 0.8700, 0.6600])
2 tensor([0.5700, 0.8500, 0.6400])
3 tensor([0.2200, 0.5800, 0.3300])
4 tensor([0.7700, 0.2500, 0.1000])
5 tensor([0.0500, 0.8000, 0.5500])


In [15]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.10], # 0ne
        [0.05, 0.80, 0.55]  # step
    ]
)

# 3.3.2 Computing attention weights for all input tokens

In [16]:


attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attn_scores[i,j] = torch.dot(x_i,x_j)



print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [17]:
attn_scores =inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=0)
all_context_vecs = attn_weights @ inputs
all_context_vecs


tensor([[0.4017, 0.5023, 0.5059],
        [0.5595, 0.7824, 0.6953],
        [0.5538, 0.7686, 0.6834],
        [0.3369, 0.4647, 0.4119],
        [0.3525, 0.4059, 0.3657],
        [0.3856, 0.5761, 0.5077]])

# 3.4 implementing self_attention with trainable weights

# 3.4.1 Computing self-attention weights step by step

In [18]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [19]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [20]:
query_2 = x_2 @ W_query
query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [21]:
keys = inputs @ W_key
value = inputs @ W_value
keys.shape

torch.Size([6, 2])

In [22]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)

In [23]:
keys_2 = keys[1]
attn_scores22 = torch.dot(query_2,keys_2)

In [24]:
attn_scores22

tensor(1.8524, grad_fn=<DotBackward0>)

In [25]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [75]:
d_k = keys.shape[1]
print(d_k)
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
attn_weights_2

2


tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

In [27]:
context_vecs_2 = attn_weights_2 @ value
context_vecs_2


tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)

# 3.4.2 Implementing a compact SelfAttention class

In [28]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

  def __init__(self, d_in, d_out):
    super().__init__()
    self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
    self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
    self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))


  def forward(self,x):
    queries = inputs @ W_query
    keys = inputs @ W_key
    values = inputs @ W_value
    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
    context_vecs = attn_weights @ values
    return context_vecs

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
sa_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

# Using Linear Layer

In [29]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):

  def __init__(self, d_in, d_out, qkv_bias=False):
    super().__init__()
    self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)


  def forward(self,x):
    queries = self.W_query(x)
    keys = self.W_key(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
    context_vecs = attn_weights @ values
    return context_vecs

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

# 3.5 Hiding future words with casual attention

# 3.5.1 Applying a causal attention mask

In [30]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
values = sa_v2.W_value(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)

In [31]:
attn_weights

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [32]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [33]:
masked_simple = attn_weights * mask_simple
masked_simple

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)

In [34]:
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
masked_simple_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)

In [35]:
masked_simple = attn_scores*mask_simple
masked_simple_nrm = torch.softmax(masked_simple / d_k**0.5, dim=-1)
masked_simple_nrm


tensor([[0.1971, 0.1606, 0.1606, 0.1606, 0.1606, 0.1606],
        [0.2132, 0.1733, 0.1534, 0.1534, 0.1534, 0.1534],
        [0.2083, 0.1698, 0.1702, 0.1506, 0.1506, 0.1506],
        [0.1892, 0.1688, 0.1689, 0.1591, 0.1570, 0.1570],
        [0.1833, 0.1671, 0.1672, 0.1591, 0.1661, 0.1571],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [79]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
print(mask)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [37]:
attn_weights = torch.softmax(masked / d_k**0.5, dim=-1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

# 3.5.2 Masking additional attention weights with dropout

In [38]:
torch.manual_seed(123)
layer = torch.nn.Dropout(0.5)


In [39]:
example = torch.ones(6,6)
example

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [40]:
layer(example)

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])

In [41]:
dropout_rate = 0.5
1 / (1 -dropout_rate)

2.0

In [42]:
layer(attn_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.0000, 0.4638, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.3968, 0.3775, 0.3941, 0.0000],
        [0.3869, 0.3327, 0.0000, 0.0000, 0.3331, 0.3058]],
       grad_fn=<MulBackward0>)

# 3.5.3 Implementing a compact causal self_attention class

In [89]:
batch = torch.stack((inputs, inputs),dim=0)
batch.shape[1]

6

In [92]:
import torch.nn as nn

class CausalAttention(nn.Module):

  def __init__(self, d_in, d_out,context_length, dropout, qkv_bias=False):
    super().__init__()
    self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = torch.nn.Dropout(dropout)
    self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))


  def forward(self,x):
    b, num_tokens, d_in = x.shape
    print(x.shape)
    # x = batch, 2*6*3
    queries = self.W_query(x)
    keys = self.W_key(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.transpose(1,2) # Changed transpose
    attn_scores.masked_fill_( # New,_ops are in-place
         self.mask.bool()[:num_tokens, :num_tokens],-torch.inf)
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5,dim=-1)
    attn_weights = self.dropout(attn_weights)
    context_vecs = attn_weights @ values

    return context_vecs



torch.manual_seed(789)
context_length = batch.shape[1]
dropout = 0.0
ca = CausalAttention(d_in, d_out, context_length, dropout_rate)
ca(batch)



torch.Size([2, 6, 3])


tensor([[[-0.1744,  0.0572],
         [-0.0962,  0.0315],
         [-0.1336,  0.1050],
         [-0.1542,  0.0991],
         [-0.0749,  0.0358],
         [-0.1636,  0.0083]],

        [[-0.1744,  0.0572],
         [ 0.0000,  0.0000],
         [-0.1367,  0.0691],
         [-0.1406,  0.0602],
         [-0.1175,  0.0418],
         [-0.1131,  0.1131]]], grad_fn=<UnsafeViewBackward0>)

# 3.6 Extending single-head attention to multi-head attention

# 3.6.1 Stacking multiple single-head attention layers

In [94]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self,d_in, d_out,context_length, dropout, num_heads=2, qkv_bias=False):
    super().__init__()
    self.heads = nn.ModuleList(
        [CausalAttention(d_in, d_out,context_length, dropout, qkv_bias) for _ in range(num_heads)])

  def forward(self, x):
    return torch.cat([head(x) for head in self.heads],dim=-1)

torch.manual_seed(123)
context_length = batch.shape[1]
d_in,d_out = batch.shape[-1],2
dropout = 0.0
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout, num_heads=2)
mha(batch)

torch.Size([2, 6, 3])
torch.Size([2, 6, 3])


tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)

# 3.6.2 Implementing multi-head attention with weights splits

In [99]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_in, d_out,context_length, dropout, num_heads=2, qkv_bias=False):
    super().__init__()
    assert(d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"

    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads # reduce the projection dim to match desired output dim

    self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    self.out_proj = torch.nn.Linear(d_out, d_out)
    self.dropout = torch.nn.Dropout(dropout)
    self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))


  def forward(self, x):
    b, num_tokens, d_in = x.shape # Shape : (b, num_tokens, d_out)
    queries = self.W_query(x)
    keys = self.W_key(x)
    values = self.W_value(x)

    # we implicitly split the matrix by adding a ` num_heads` dimention
    # unroll llast dim : (b, num_tokens, d_out) -->(b, num_tokens,  head_dim)

    keys = keys.reshape(b, num_tokens, self.num_heads, self.head_dim)
    queries = queries.reshape(b, num_tokens, self.num_heads, self.head_dim)
    values = values.reshape(b, num_tokens, self.num_heads, self.head_dim)

    #Transpose: ((b, num_tokens, num_heads, head_dim) -->(b, num_heads, num_tokens,  head_dim)
    keys = keys.transpose(1,2)
    queries = queries.transpose(1,2)
    values = values.transpose(1,2)

    # Compute scaled dot-product attention (selfattention) with a causal mask

    attn_scores = queries @ keys.transpose(2,3) # dot product for each head

    # Original mask truncated to the number of tokens and converted to boolean

    mask_bool = self.mask[:num_tokens, :num_tokens].bool()

    # Use the mask to fill attention scores
    attn_scores.masked_fill_(mask_bool, -torch.inf)
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5,dim=-1)
    attn_weights = self.dropout(attn_weights)

    # Shape: (b, num_tokens, num_heads, head_dim)
    context_vec = (attn_weights @ values).transpose(1,2)

    # Combine heads, where self.d_out = self.num_heads * self.head_dim
    context_vec = context_vec.reshape(b, num_tokens, self.d_out)

    contex_vec = self.out_proj(context_vec) # optional projection

    return context_vec

torch.manual_seed(123)

batch__size, context_length, d_in = batch.shape
d_out = 4

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
mha(batch)





tensor([[[-0.3132, -0.2272,  0.4772,  0.1063],
         [-0.2308,  0.0329,  0.5764,  0.3007],
         [-0.2059,  0.1190,  0.6097,  0.3654],
         [-0.1642,  0.1340,  0.5431,  0.3503],
         [-0.1689,  0.1794,  0.5296,  0.3389],
         [-0.1407,  0.1699,  0.5040,  0.3403]],

        [[-0.3132, -0.2272,  0.4772,  0.1063],
         [-0.2308,  0.0329,  0.5764,  0.3007],
         [-0.2059,  0.1190,  0.6097,  0.3654],
         [-0.1642,  0.1340,  0.5431,  0.3503],
         [-0.1689,  0.1794,  0.5296,  0.3389],
         [-0.1407,  0.1699,  0.5040,  0.3403]]], grad_fn=<UnsafeViewBackward0>)